In [55]:
from web3 import Web3, HTTPProvider, IPCProvider, WebsocketProvider
import json
import pandas as pd
from datetime import datetime
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


In [12]:
web3 = Web3(HTTPProvider('https://ropsten.infura.io/v3/831ab04fa4964991b5fba5c52106d7b0'))

### ASCII & HASH

In [67]:
text = "bas"
print(Web3.toHex(text = text))
print(Web3.keccak(hexstr = Web3.toHex(text = text)).hex())

0x626173
0x1afc1e6d002095d67ae3d63e7ab2500b3675166cbe7353af950c3c2e82cec592


### ASCII to String

In [14]:
web3.toText( 0x73756e7a79)

'sunzy'

### Recover Contracts by BasRelations

In [18]:
BasRelations_addr = "0x3cD2E8fA4167232298C7d332A46b78FA937959F2"
BasView_addr = "0xf3e0222FC99897E3569F4490026D914A9421572a"

with open("./abi/BasRelations.abi") as f:
    BasRelations = web3.eth.contract(address=BasRelations_addr, abi=json.load(f))
    
with open("./abi/BasView.abi") as f:
    BasView = web3.eth.contract(address=BasView_addr, abi=json.load(f))

with open("./abi/BasToken.abi") as f:
    BasToken = web3.eth.contract(address=BasRelations.functions.token().call(), abi=json.load(f))

with open("./abi/BasExpiredOwnership.abi") as f:
    BasExpiredOwnership = web3.eth.contract(address=BasRelations.functions.exo().call(), abi=json.load(f))

with open("./abi/BasTradableOwnership.abi") as f:
    BasTradbleOwnership = web3.eth.contract(address=BasRelations.functions.tro().call(), abi=json.load(f))

with open("./abi/BasRootDomain.abi") as f:
    BasRootDomain = web3.eth.contract(address=BasRelations.functions.root().call(), abi=json.load(f))
    
with open("./abi/BasSubDomain.abi") as f:
    BasSubDomain = web3.eth.contract(address=BasRelations.functions.sub().call(), abi=json.load(f))
    
with open("./abi/BasDomainConf.abi") as f:
    BasDomainConf = web3.eth.contract(address=BasRelations.functions.conf().call(), abi=json.load(f))
    
with open("./abi/BasAccountant.abi") as f:
    BasAccountant = web3.eth.contract(address=BasRelations.functions.acc().call(), abi=json.load(f))
    
with open("./abi/BasMiner.abi") as f:
    BasMiner = web3.eth.contract(address=BasRelations.functions.miner().call(), abi=json.load(f))
    
with open("./abi/BasOANN.abi") as f:
    BasOANN = web3.eth.contract(address=BasRelations.functions.oann().call(), abi=json.load(f))
    
with open("./abi/BasMarket.abi") as f:
    BasMarket = web3.eth.contract(address=BasRelations.functions.market().call(), abi=json.load(f))
    
with open("./abi/BasMail.abi") as f:
    BasMail = web3.eth.contract(address=BasRelations.functions.mail().call(), abi=json.load(f))
    
with open("./abi/BasMailManager.abi") as f:
    BasMailManager = web3.eth.contract(address=BasRelations.functions.mm().call(), abi=json.load(f))

### Account Address

In [49]:
Account = "0xC5DA9e80e6d27E4bf14d961EE50bA8596b3974De"

### Query Functions

In [196]:
def convertTime(uinx_time):
    return datetime.utcfromtimestamp(uinx_time).strftime('%Y-%m-%d')

def GetDomainInfos(account):
    domains = BasTradbleOwnership.functions.assetsOf(0,0).call({"from":Account})
    info = list(map(lambda x: BasView.functions.queryDomainInfo(x.hex()).call(),domains))
    df = pd.DataFrame(columns=["name","owner","expiration","isRoot","rIsOpen","rIsCustomed","rIsRare","rCusPrice","sRootHash","isMarketOrder"],data = info)
    df["expiration"] = df["expiration"].map(lambda x : convertTime(x))
    df["name"] = df["name"].map(lambda x : web3.toText(x))
    df["sRootHash"] = df["sRootHash"].map(lambda x : x.hex())
    return df

def GetDomainEmailInfos(account):
    domains = BasTradbleOwnership.functions.assetsOf(0,0).call({"from":Account})
    info = list(map(lambda x: BasView.functions.queryDomainEmailInfo(x.hex()).call(),domains))
    df = pd.DataFrame(columns=["name","owner","expiration","isActive","openToPublic"],data = info)
    df["expiration"] = df["expiration"].map(lambda x : convertTime(x))
    df["name"] = df["name"].map(lambda x : web3.toText(x))
    return df

def GetEmailInfos(account):
    domains = BasExpiredOwnership.functions.assetsOf(0,0).call({"from":Account})
    hashes = pd.DataFrame(columns = ["mailHash"], data = domains)
    hashes["mailHash"] = hashes["mailHash"].map(lambda x: x.hex())
    info = list(map(lambda x: BasView.functions.queryEmailInfo(x.hex()).call(),domains))
    df = pd.DataFrame(columns=["owner","expiration","domainHash","isValid","aliasName","bcAddress"],data = info)
    df["domainHash"] = df["domainHash"].map(lambda x : x.hex())
    df["aliasName"] = df["aliasName"].map(lambda x : web3.toText(x))
    df["bcAddress"] = df["bcAddress"].map(lambda x : web3.toText(x))
    df["expiration"] = df["expiration"].map(lambda x : convertTime(x))
    df = df[["aliasName","owner","expiration","domainHash","isValid","bcAddress"]]
    df = pd.concat([hashes, df], axis= 1)
    return df

def GetAllRootDomains():
    d = BasRootDomain.events.NewRootDomain.getLogs(fromBlock=0,toBlock="latest")
    d = list(map(lambda x : x["args"],d))
    df = pd.DataFrame(columns=["nameHash","rootName","openToPublic","isCustom","customPrice"],data = d)
    df = df[["rootName","nameHash"]]
    df["nameHash"] = df["nameHash"].map(lambda x : x.hex())
    df["rootName"] = df["rootName"].map(lambda x : web3.toText(x))
    return df

def GetAllSubDomains():
    d = BasSubDomain.events.NewSubDomain.getLogs(fromBlock=0,toBlock="latest")
    d = list(map(lambda x : x["args"],d))
    df = pd.DataFrame(columns=["nameHash","totalName","rootHash"],data = d)
    df = df[["totalName","nameHash", "rootHash"]]
    df["nameHash"] = df["nameHash"].map(lambda x : x.hex())
    df["rootHash"] = df["rootHash"].map(lambda x : x.hex())
    df["totalName"] = df["totalName"].map(lambda x : web3.toText(x))
    return df

def GetAllDomains():
    root = GetAllRootDomains()
    root.columns = ["name","hash"]
    sub = GetAllSubDomains()
    sub.columns = ["name", "hash", "root"]
    root["type"] = "root"
    sub["type"] = "sub"
    return pd.concat([root,sub])[["type","name","hash"]]

def GetAllDomainsWithOwner():
    d = GetAllDomains()
    owner_expire = d["hash"].map(lambda x : BasTradbleOwnership.functions.ownerOfWithExpire(x).call())
    owner_expire = list(map(lambda x : [x[0],x[1]], owner_expire))
    d2 = pd.DataFrame(columns=["owner","expiration"], data= owner_expire)
    return pd.concat([d.reset_index(),d2],axis=1).drop(["index"],axis=1)


In [110]:
GetDomainInfos(Account)

,name,owner,expiration,isRoot,rIsOpen,rIsCustomed,rIsRare,rCusPrice,sRootHash,isMarketOrder
0,bmail1,0xC5DA9e80e6d27E4bf14d961EE50bA8596b3974De,2021-05-22,True,True,False,True,4000000000000000000,0000000000000000000000000000000000000000000000000000000000000000,False
1,bmail2,0xC5DA9e80e6d27E4bf14d961EE50bA8596b3974De,2021-05-22,True,True,False,True,4000000000000000000,0000000000000000000000000000000000000000000000000000000000000000,False
2,bmail,0xC5DA9e80e6d27E4bf14d961EE50bA8596b3974De,2021-05-22,True,True,False,True,4000000000000000000,0000000000000000000000000000000000000000000000000000000000000000,False


In [111]:
GetDomainEmailInfos(Account)

,name,owner,expiration,isActive,openToPublic
0,bmail1,0xC5DA9e80e6d27E4bf14d961EE50bA8596b3974De,2021-05-22,False,False
1,bmail2,0xC5DA9e80e6d27E4bf14d961EE50bA8596b3974De,2021-05-22,False,False
2,bmail,0xC5DA9e80e6d27E4bf14d961EE50bA8596b3974De,2021-05-22,True,True


In [112]:
GetEmailInfos(Account)

,mailHash,aliasName,owner,expiration,domainHash,isValid,bcAddress
0,50cd69341246ab394b421b98ed926e6f3b679444edcf057b1a10d108e48c9857,sunzy@bmail,0xC5DA9e80e6d27E4bf14d961EE50bA8596b3974De,2021-05-22,9340a8b9bee032c268c2a65cf10e7ae1ffc7e1b184903e1702fae2fefc80642f,True,


In [124]:
GetAllRootDomains()

,rootName,nameHash
0,eth,4f5b812789fc606be1b3b16908db13fc7a9adf7ca72641f84d75b47069d3d7f0
1,btc,4bac7d8baf3f4f429951de9baff555c2f70564c6a43361e09971ef219908703d
2,lanbery,00f69be5c125d6ec3023374357cb911088ab1a7e72b3f9d1eb5fc68dc3e1aa1a
3,bmail2,54f3c30da7371c46911076eb1e249a270f181184a576a4a6d5fac820e575c567
4,yak,233611481cca65c3f839ffff0bed03c6bbaccd8de66208c89a057e9072226d16
5,bmail,9340a8b9bee032c268c2a65cf10e7ae1ffc7e1b184903e1702fae2fefc80642f
6,youtube,fe50d0d4db7231b94a5b6037ab2fc77cf3ead9a7e85f46a61faa66a815fed91d
7,bas,1afc1e6d002095d67ae3d63e7ab2500b3675166cbe7353af950c3c2e82cec592


In [127]:
GetAllSubDomains()

,totalName,nameHash,rootHash
0,cwd.bas,66df287be06e92d94266b623539fdc4e4bf614d2e2b0b7bee991223af635dc04,1afc1e6d002095d67ae3d63e7ab2500b3675166cbe7353af950c3c2e82cec592
1,xn--qei.bas,439c3fce5deb731f85f048903b8c737e98810b19a96c1422e2efe9b35931da62,1afc1e6d002095d67ae3d63e7ab2500b3675166cbe7353af950c3c2e82cec592
2,lanbery.bas,26dd2376565207caa14cfa4628067357e262a2ed5a726b6671da0c07957ee574,1afc1e6d002095d67ae3d63e7ab2500b3675166cbe7353af950c3c2e82cec592
3,kill.bas,8c9bf3e666e0d03ee4272db2329a05c644db0c1a839201ec6c4989938558f5a6,1afc1e6d002095d67ae3d63e7ab2500b3675166cbe7353af950c3c2e82cec592
4,okex.eth,4b2315e65a9414dd84e8fcfeb28f4cf1d3c179e521be4367ed7c2eebc30304e2,4f5b812789fc606be1b3b16908db13fc7a9adf7ca72641f84d75b47069d3d7f0
5,tech.btc,fbf1b9ca32f5a7fd6d0e138798067ab58ae64553d7b117c9a680d2148cf919f9,4bac7d8baf3f4f429951de9baff555c2f70564c6a43361e09971ef219908703d


In [197]:
GetAllDomainsWithOwner()

,type,name,hash,owner,expiration
0,root,eth,4f5b812789fc606be1b3b16908db13fc7a9adf7ca72641f84d75b47069d3d7f0,0x3D002404deee63697fBEf95657DcE57335BF561D,1621659303
1,root,btc,4bac7d8baf3f4f429951de9baff555c2f70564c6a43361e09971ef219908703d,0xc0E6a6F90D4Cd02F13447565171Fea4bf91D73A8,1621659984
2,root,lanbery,00f69be5c125d6ec3023374357cb911088ab1a7e72b3f9d1eb5fc68dc3e1aa1a,0xFd30d2c32E6A22c2f026225f1cEeA72bFD9De865,1621662360
3,root,bmail2,54f3c30da7371c46911076eb1e249a270f181184a576a4a6d5fac820e575c567,0xC5DA9e80e6d27E4bf14d961EE50bA8596b3974De,1621665469
4,root,yak,233611481cca65c3f839ffff0bed03c6bbaccd8de66208c89a057e9072226d16,0xea8a3a416799d582bC46987E084886524E7449Df,1621673946
5,root,bmail,9340a8b9bee032c268c2a65cf10e7ae1ffc7e1b184903e1702fae2fefc80642f,0xC5DA9e80e6d27E4bf14d961EE50bA8596b3974De,1621675080
6,root,youtube,fe50d0d4db7231b94a5b6037ab2fc77cf3ead9a7e85f46a61faa66a815fed91d,0xFd30d2c32E6A22c2f026225f1cEeA72bFD9De865,1621695026
7,root,bas,1afc1e6d002095d67ae3d63e7ab2500b3675166cbe7353af950c3c2e82cec592,0xea8a3a416799d582bC46987E084886524E7449Df,1621734081
8,sub,cwd.bas,66df287be06e92d94266b623539fdc4e4bf614d2e2b0b7bee991223af635dc04,0xFd30d2c32E6A22c2f026225f1cEeA72bFD9De865,1621659388
9,sub,xn--qei.bas,439c3fce5deb731f85f048903b8c737e98810b19a96c1422e2efe9b35931da62,0xFd30d2c32E6A22c2f026225f1cEeA72bFD9De865,1621659388
